In [1]:
%reload_kedro

2023-05-08 16:36:16,701 - kedro.extras.extensions.ipython - INFO - No path argument was provided. Using: /Users/gpalazzo/Desktop/dev/crypto_msc_thesis
2023-05-08 16:36:17,227 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2023-05-08 16:36:17,359 - kedro.extras.extensions.ipython - INFO - ** Kedro project crypto_thesis
2023-05-08 16:36:17,360 - kedro.extras.extensions.ipython - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`


In [2]:
from crypto_thesis.data_domains.modeling import logreg_model_fit, lstm_model_fit, xgboost_model_fit
from typing import Dict, List
import itertools
import time
import numpy as np

In [3]:
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
tf.get_logger().setLevel("ERROR") #only show error messages

## Base

In [4]:
mt_multic = catalog.load("master_table_multic")
mt_nonmultic = catalog.load("master_table_nonmultic")
trainTest_cutoff_date = catalog.load("params:train_test_cutoff_date")

2023-05-08 16:36:18,182 - kedro.io.data_catalog - INFO - Loading data from `master_table_multic` (ParquetDataSet)...
2023-05-08 16:36:18,300 - kedro.io.data_catalog - INFO - Loading data from `master_table_nonmultic` (ParquetDataSet)...
2023-05-08 16:36:18,308 - kedro.io.data_catalog - INFO - Loading data from `params:train_test_cutoff_date` (MemoryDataSet)...


In [5]:
def build_nbr_combinations(grid: Dict[str, List]) -> int:
    keys, values = zip(*grid.items())
    combs = itertools.product(*values)
    return list(combs)

## XGBoost

In [ ]:
def build_xgboost_param_combinations():
    return {
    'eval_metric': ['auc'],
     'n_estimators': [300, 500, 1000],
     'max_depth': [3, 5],
     'reg_lambda': [0.05, 0.01, 0.1],
     'gamma': [0.01],
     'min_child_weight': [2.0],
     'learning_rate': [0.01, 0.05, 0.1],
     'subsample': [0.7],
     'colsample_bytree': [0.5],
     'objective': ['binary:logistic'],
     'sampling_method': ['uniform'],
     'tree_method': ['auto']
    }

In [ ]:
xgb_model_params = catalog.load("params:xgboost_model_params")
xgb_def_params = catalog.load("params:xgboost_default_params")

# if True, then optimize hyperparameters
# if False, then not
xgb_opt_params = catalog.load("params:xgboost_optimize_params")
xgb_opt_params = True

print()
if xgb_opt_params:
    xgb_model_params = build_xgboost_param_combinations()
    combinations = build_nbr_combinations(grid=xgb_model_params)
    print(f"--> Optimizing hyperparameters with total combinations of: {len(combinations)}")
else:
    print("--> NOT optimizing hyperparameters")

In [ ]:
%%timeit
_, _, _, _, _, _ = xgboost_model_fit(master_table=mt_multic, 
                                     train_test_cutoff_date=trainTest_cutoff_date, 
                                     model_params=xgb_model_params, 
                                     xgboost_optimize_params=xgb_opt_params, 
                                     xgboost_default_params=xgb_def_params)

## LSTM

In [ ]:
lstm_timestamp_seq_length = catalog.load("params:lstm_timestamp_seq_length")

# if True, then optimize hyperparameters
# if False, then not
lstm_opt_params = False

print()
if lstm_opt_params:
    print("--> Optimizing hyperparameters")
else:
    print("--> NOT optimizing hyperparameters")

In [ ]:
%%timeit
_, _, _, _, _, _ = lstm_model_fit(master_table=mt_multic, 
                                     train_test_cutoff_date=trainTest_cutoff_date, 
                                     seq_length=lstm_timestamp_seq_length)

## LogReg

In [6]:
def build_logreg_param_combinations():
    return {
      "solver": ["saga"],
      "penalty": ["elasticnet"],
      "tol": [0.0001, 0.001, 0.01],
      "C": [0.01, 0.1, 1.0],
      "max_iter": [100, 200],
      "fit_intercept": [True],
      "class_weight": ["balanced"],
      "l1_ratio": [0.01, 0.1, 1.0]
    }

In [7]:
logreg_model_params = catalog.load("params:logreg_model_params")
logreg_def_params = catalog.load("params:logreg_default_params")

# if True, then optimize hyperparameters
# if False, then not
logreg_opt_params = catalog.load("params:logreg_optimize_params")
logreg_opt_params = True

print()
if logreg_opt_params:
    logreg_model_params = build_logreg_param_combinations()
    combinations = build_nbr_combinations(grid=logreg_model_params)
    print(f"--> Optimizing hyperparameters with total combinations of: {len(combinations)}")
else:
    print("--> NOT optimizing hyperparameters")

2023-05-08 16:36:22,566 - kedro.io.data_catalog - INFO - Loading data from `params:logreg_model_params` (MemoryDataSet)...
2023-05-08 16:36:22,568 - kedro.io.data_catalog - INFO - Loading data from `params:logreg_default_params` (MemoryDataSet)...
2023-05-08 16:36:22,569 - kedro.io.data_catalog - INFO - Loading data from `params:logreg_optimize_params` (MemoryDataSet)...

--> Optimizing hyperparameters with total combinations of: 54


In [8]:
times = []

for i in range(0, 5):
    start = time.time()
    _, _, _, _, _, _ = logreg_model_fit(master_table=mt_nonmultic, 
                                     train_test_cutoff_date=trainTest_cutoff_date, 
                                     model_params=logreg_model_params, 
                                     logreg_optimize_params=logreg_opt_params, 
                                     logreg_default_params=logreg_def_params)
    end = time.time()
    
    times.append(end - start)

2023-05-08 16:36:23,279 - crypto_thesis.data_domains.modeling.logreg - INFO - Optimzing parameters


/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/gpalazzo/anaconda3/envs/m

/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/gpalazzo/anaconda3/envs/m

/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/gpalazzo/anaconda3/envs/m

/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/gpalazzo/anaconda3/envs/m

...

/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/gpalazzo/anaconda3/envs/m

/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/gpalazzo/anaconda3/envs/m

/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gpalazzo/anaconda3/envs/m

/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/gpalazzo/anaconda3/envs/msc/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was 

In [9]:
print(f"Média (s): {np.mean(times)}\nDesvio (s): {np.std(times, ddof=1)}")

Média (s): 35.269085788726805
Desvio (s): 7.520074865166375
